In [1]:
from emnist import extract_training_samples
images, labels = extract_training_samples('letters')
images.shape


(124800, 28, 28)

In [ ]:
##Convert all images to .jpg format and saved in a folder.
from PIL import Image
import numpy as np
import os
import cv2
def gen_image(arr):
    two_d = np.reshape(arr, (28, 28))
    img = Image.fromarray(two_d, 'L')
    return img
save_train_dir = './train_images/'
if not os.path.exists(save_train_dir):
        os.makedirs(save_train_dir)
for i in range(len(images)):
#     image_list = [int(item) for item in image_file[i * 784: i*784+784]] # Convert binary data to decimal
    image_array = np.array(images[i],dtype=np.uint8).reshape(28,28,1)
    label_name = labels[i]
    image_name = os.path.join(save_train_dir,str(i+1)+'_'+str(label_name)+'.jpg')
#     print(image_name)
    cv2.imwrite(image_name,image_array)
    pass
print(os.listdir(save_train_dir))
    


In [2]:
# Convert all images stored in the local to a sprie sheet. 
from pathlib import Path
from PIL import Image
import numpy as np

# Constants
TRAINING_PATH = "./train_images/"
SPRITE_SIZE = 28

# Initialization
x_data = []
y_data = []
final_image = np.array([])
y_offset = 0
new_im = Image.new('RGB', (SPRITE_SIZE*SPRITE_SIZE, 124800))

# Load the training sprite by looping over every image file
for image_file in Path(TRAINING_PATH).glob("**/*.jpg"):

    # Load the current image file
    src_image = Image.open(image_file)
    # make it smaller
    downsized = src_image.resize((SPRITE_SIZE,SPRITE_SIZE)) 

    # get 1px high version
    pixels = list(downsized.getdata())
    smoosh = Image.new('RGB', (SPRITE_SIZE * SPRITE_SIZE, 1))
    smoosh.putdata(pixels)

    # store image
    x_data.append(smoosh)

    # Use image path to build our answer key
    y_data.append(image_file.stem.split('_')[1])
print(x_data[0],y_data[0])
   
# Now randomize X and Y the same way before making data
# (the JS code splits then randomizes) DERP!!!
assert len(y_data) == len(x_data)
p = np.random.permutation(len(y_data))
npy = np.array(y_data)
shuffled_y = npy[p].tolist()

one_hot_y = []
# Build the data image and 1-hot encoded answer array
for idx in p:
    # build master sprite 1 pixel down at a time
    new_im.paste(x_data[idx], (0, y_offset))
    # build 1-hot encoded answer key
    for i in range(26):
        if shuffled_y[y_offset] == str(i+1):
            one_hot_y.append(1)
        else:
            one_hot_y.append(0)
    y_offset += 1        


# Save answers file (Y)
newFile = open("labels_img", "wb")
newFileByteArray = bytearray(one_hot_y)
bytesWritte = newFile.write(newFileByteArray)
# should be num classes * original answer key size
assert bytesWritte == (26 * len(y_data))

# Save Data Sprite (X)
new_im.save('data_img.png')


<PIL.Image.Image image mode=RGB size=784x1 at 0x1F4DD39D908> 12
